# Approximate Nearest Neighbors:

# Image Recommendation System via Collaborative Filtering

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ***Please read the instructions very carefully***
This is a modified version of the previous question and requires you to use an artificial nearest neighbors library

We suggest you to use one of the following:
- [ScaNN](https://github.com/google-research/google-research/tree/master/scann)
- [FAISS](https://github.com/facebookresearch/faiss)
- [Annoy](https://github.com/spotify/annoy.git)

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

You may use any other recommendation system that you wish to use. However,
evaluation script will score your submission by measuring the similarity between
users with the number of common images they appreciated.
The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

### Import necessary libraries

In [3]:
%pip install tensorflow
%pip install scann

     |████████████████████████████████| 10.6 MB 4.7 MB/s 


In [4]:
import numpy as np
import pandas as pd
import os, time, sys, scann
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix

In [5]:
config = pd.read_csv('config.csv').iloc[0]
# config = {'id':276633,
#           'k':5,
#           'dataset_file':'./Behance_appreciate_1M',
#           'output_file':'./output.txt'}

In [6]:
config

{'dataset_file': './Behance_appreciate_1M',
 'id': 276633,
 'k': 5,
 'output_file': './output.txt'}

In [7]:
user = config['id']
k_value = config['k']

### Read the Data

In [8]:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()
# with open('/content/drive/MyDrive/Behance_appreciate_1M', 'r') as inFile:
#     appreciate_data = inFile.readlines()

In [9]:
# your code here

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [10]:
user_likes = dict()

In [11]:
userData = dict()                   # userData is the final preprocessed data
users_dict = dict()
images_dict = dict()
processed_data = np.empty((63497, 178788), dtype=bool)

for line in appreciate_data:
  val = line.split(' ')
  user = int(val[0])
  image = int(val[1])

  if user not in users_dict:
    users_dict[user] = len(users_dict)
    userData[users_dict[user]] = []
  user_id = users_dict[user]
  if image not in images_dict:
    images_dict[image] = len(images_dict)
  image_id = images_dict[image]

  userData[user_id].append(image)
  processed_data[user_id][image_id] = True


denseMatrix = csr_matrix(processed_data, dtype=float)
processed_data = 0
reducedMatrix, s, vt = svds(denseMatrix, k = 1000)

In [12]:
def compute_recall(neighbors, true_neighbors):
    total = 0
    for gt_row, row in zip(true_neighbors, neighbors):
        total += np.intersect1d(gt_row, row).shape[0]
    return total / true_neighbors.size

### Use your choice of Approximate Nearest Neigbor after Collaborative Filtering to find nearest neighbors

In [13]:
# your code here
normalized_dataset = reducedMatrix / np.linalg.norm(reducedMatrix, axis=1)[:, np.newaxis]

searcher = scann.scann_ops_pybind.builder(normalized_dataset, 10, "dot_product").tree(
    num_leaves=2000, num_leaves_to_search=100, training_sample_size=reducedMatrix.shape[0]).score_ah(
    2, anisotropic_quantization_threshold=0.2).reorder(100).build()
def neighbors(user,k_value):
  global normalized_dataset, searcher
  
  queries = [user]
  neighbors, distances = searcher.search(queries, leaves_to_search=150)
  return neighbors[0,:k_value]



In [14]:
def neighbors(user,k_value):
  global normalized_dataset, searcher, users_dict
  
  neighbors, distances = searcher.search(normalized_dataset[users_dict[user]], leaves_to_search=150)
  return neighbors[:k_value]

In [15]:
print(neighbors(user, k_value))

[63496 33929 38741 41294 42540]


### Answer the following questions:

#### Q1. **Explain how your choice of library works**
The library that I have employed in this question is Google's **Scalable Nearest Neighbors** (ScaNN). It is one of the most recent advancements in KNN algorithm and performs the best for large number of queries (the usecase that google needed it for).

It uses *maximum inner-product* (MIPS) as the distance metric. It uses a new solution to compress the embeddings which is to quantize the vectors into one of certain allowed representations thereby decreasing the search space. The quantization is performed by *anisotropic vector quantization* where in a vector is quantized to another vector such that it has the least loss in the direction of the data-point x, not the one which is closest to it. This way, the MIPS is optimized to decrease the time per query with the minimal loss of information.


#### Q2. **Compare your choice of library with vanilla KNN.**
***Hint: Include Time Complexity, and explain the tradeoff with recall***


### Ans


A normal KNN algorithm (aka vanilla KNN) computes distance between all pair of points. This ensures the best abidance to KNN algorithm but is very very slow for practical purposes. This solution offers best recall but the tradeoff with speed is very poor.


On the other hand, ScaNN has a good speed-recall tradeoff since it uses approximations to lower the distance calculation cost. It uses vector quantization with minimal loss of information. Due to this same reason, it is fairly accurate and scalable as well

#### Q3. **Compare your choice of library with implementation of ScaNN, faiss and annoy.**
***Hint: Include Time Complexity, and explain the tradeoff with recall***



#### Ans


The chosen implementation is ScaNN. It uses anisotropic vector quantization to achieve better vector approximations. But this method does not save it any time. The reason why ScaNN is the fastest and most accurate KNN algorithm for public use is that it uses in-register lookups for saving time. This also allows it to have a better trade-off with recall. The following graphs shows it.


![alt text for screen readers](tradeoff.png)


Comparing ScaNN with Annoy (by Spotify), ScaNN is not exceptionally fast because it has a very large set of indexes (or embeddings) hence it chooses to optimize on solving the problem of the size of index files. 

It has a system of having a tree structure for indexes and allowing multiple processes to use the same set of indexes allowing parallel computing. Visibly, the speed tradeoff with recall is not so much as good as ScaNN, but that is only due to a difference in the usecase (Sptofity, Uber, Instacart don't use it for nothing!)

### Open the output file to write all the lines to the file

In [26]:
outFile = open(config['output_file'], 'w')

for n_user in neighbors(user,k_value):
    user_id = n_user
    real_id = 0
    for item in users_dict:
      if users_dict[item] == user_id:
        real_id = item
        break
    for item_id in userData[user_id]:
        if item_id not in userData[users_dict[user]]:
            outFile.write(str(item_id) + ' ' + str(real_id) + '\n')

outFile.close()